In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os
import time
import numpy as np

In [18]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print()
print(tf.__version__)

Num GPUs Available:  1

2.7.0


In [19]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                   )
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='categorical')

Found 8000 images belonging to 2 classes.


In [20]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='categorical')

Found 2000 images belonging to 2 classes.


In [37]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
print(cnn.output_shape)
cnn.add(tf.keras.layers.Flatten())
print(cnn.output_shape)
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
print(cnn.output_shape)
cnn.add(tf.keras.layers.Dense(units=2, activation="sigmoid"))
cnn.add(tf.keras.layers.Softmax())

(None, 14, 14, 32)
(None, 6272)
(None, 128)


In [38]:
cnn.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [39]:
t = time.process_time()
cnn.fit(x=training_set, validation_data=test_set, epochs=100, callbacks=[early_stopping])
elapsed_time = time.process_time() - t
print()
print()
print()
print("____________________________________________")
print(elapsed_time)
print("____________________________________________")
print()
print()
print()

Epoch 1/100
250/250 [==============================] - 17s 66ms/step - loss: 0.6869 - accuracy: 0.5422 - val_loss: 0.6583 - val_accuracy: 0.6265
Epoch 2/100
250/250 [==============================] - 17s 66ms/step - loss: 0.6545 - accuracy: 0.6110 - val_loss: 0.6156 - val_accuracy: 0.6780
Epoch 3/100
250/250 [==============================] - 17s 67ms/step - loss: 0.6154 - accuracy: 0.6634 - val_loss: 0.6021 - val_accuracy: 0.6925
Epoch 4/100
250/250 [==============================] - 17s 67ms/step - loss: 0.6079 - accuracy: 0.6749 - val_loss: 0.5905 - val_accuracy: 0.7005
Epoch 5/100
250/250 [==============================] - 17s 68ms/step - loss: 0.5989 - accuracy: 0.6891 - val_loss: 0.5990 - val_accuracy: 0.6825
Epoch 6/100
250/250 [==============================] - 17s 67ms/step - loss: 0.5849 - accuracy: 0.7067 - val_loss: 0.5773 - val_accuracy: 0.7135
Epoch 7/100
250/250 [==============================] - 17s 67ms/step - loss: 0.5804 - accuracy: 0.7090 - val_loss: 0.5812 - val_ac

In [19]:
cnn.evaluate(test_set, verbose=2)

63/63 - 3s - loss: 0.5021 - accuracy: 0.7960 - 3s/epoch - 40ms/step


[0.5020958185195923, 0.7960000038146973]

In [33]:
test_image = image.load_img('dataset/test_set/dogs/dog.4115.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)

[[0.26894143 0.73105854]]
